Importamos las librerías requeridas,

Utilizaremos:

    Pandas para trabajar con los Dataframes

    Datetime para trabajar con las fechas

    Requests para solicitar información a las url

    BeautifulSoup para interpretar la información en formato html.

In [1]:
import pandas as pd

from datetime import datetime, timedelta

import requests

from bs4 import BeautifulSoup

Primero generamos una lista con las fechas de las cuales queremos obtener datos de la url,

La intención es obtener datos desde el 19/08/2015, es la fecha hasta la que tenemos información en la base de datos ya adquirida.

In [2]:
# Establecemos la fecha inicial:
fecha_inicial = datetime(2015, 8, 19)

# Para establecer la fecha actual:
fecha_actual = datetime.now()

# Para crear el listado de fechas:
fechas = []
delta = timedelta(days=1)  # Establecemos un incremento diario.

# Para generar el listado de fechas:
while fecha_inicial <= fecha_actual:
    fechas.append(fecha_inicial)
    fecha_inicial += delta

El código que se muestra a continuación es el utilizado para hacer WebScraping de la url.

La url tiene la forma https://www.sismologia.cl/sismicidad/catalogo/YYYY/mm/YYYYmmdd.html

donde YYYY representa el año, mm el mes y dd el día.

Esta estructura es la que iteraremos con la lista de fechas obtenida en el paso anterior.

En este paso hacemos uso de Requests y de BeautifulSoup para adquirir la información e interpretarla.


In [3]:
#Creamos una lista vacía para los datos y establecemos la ruta de la url raíz:
data = []
url = "https://www.sismologia.cl/sismicidad/catalogo/"

#Iteramos fecha a fecha de la lista fechas creada en el paso anterior:
for fecha in fechas:
    # Realizar una solicitud HTTP a la página web con el formato de fecha de la url:
    num = fecha.strftime('%Y/%m/%Y%m%d.html')
    dir = url + num
    response = requests.get(dir)

    # Para crear un objeto BeautifulSoup a partir del contenido HTML obtenido:
    soup = BeautifulSoup(response.content, "html.parser")

    # Para extraer los datos de las filas:
    data_rows = soup.find_all('tr')[1:]  # Para ignorar la primera fila que contiene los encabezados.
    for row in data_rows:
        row_data = [cell.text for cell in row.find_all('td')]
        data.append(row_data)#Para agregar la información de cada fila a la lista vacía creada al comienzo.

# Para crear el DataFrame con la información almacenada en Data:
SismosChileCrudo = pd.DataFrame(data, columns=[1,2,3,4,5])

Obtenida la información, la almacenamos en formato CSV para transformarla y concatenarla al otro Dataset, pre 2015.

In [4]:
#Para almacenar la información obtenida en un archivo csv:
SismosChileCrudo.to_csv('Datasets/SismosChile2015-2023.csv', index=False)